In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from matplotlib import pyplot as plt
import numpy as np

In [3]:
import GPy, pickle

In [4]:
import pandas as pd

In [14]:
import time
import gc

In [6]:
import sys
sys.path.insert(0, '../source')

In [7]:
from CPoE_script_real import SCRIPT1, meanPD, sdPD, sdmPD

This notebook compares CPoE and non-GP methods for the real world datasets.

In [ ]:
# IMPORTANT: We do not provide the dataset directly here due to non-authoship reasons. 
# This means, you have to run first download_data.ipynb so that the datasets are available 
# in the folder datasets.

In [10]:
%run ./download_data.ipynb

# set parameters

In [8]:
# number of repetitions of training/test data splits
Nrep = 2       # for the experiment in the paper, we used Nrep = 10, which takes quite some time

In [9]:
# choose name of dataset
dataset_names = ['concrete', 'mg', 'abalone', 'space_ga', 'kin8nm']
name = dataset_names[0]

In [10]:
# path to datasets (run download_data.ipynb before) and location to store the results
path = 'datasets/'
path_results = 'results/'

In [17]:
# set parameters for each dataset

# sparsity parameter
p = 1    

# degree of correlation
PPs = [0,1,2,3]

# K0: number of experts
# MMs: number of inducing points for sparse GPs

args = {'path_results':path_results}
kks = [1,3]
if name=='concrete':
    K0 = 2**2
if name=='mg':
    K0 = 2**3
if name=='abalone':
    K0 = 2**4
if name=='space_ga':
    K0 = 2**2
if name=='kin8nm':
    K0 = 2**4
    kks = [1,2]
    args.update( {'NtestFIX':True, 'Ntestmax':3000} )

# run non-GP methods

In [15]:
SCRIPT = SCRIPT1(path+'DAT'+name+'.csv', Nreps=Nrep, name=name, **args)

resMLP, _ = SCRIPT.runNON_GP(ALG='MLP', nam='100_100', hidden_layer_sizes=(100,100))
resMLP, _ = SCRIPT.runNON_GP(ALG='MLP', nam='500_500', hidden_layer_sizes=(500,500))
resMLP, _ = SCRIPT.runNON_GP(ALG='MLP', nam='100_100_100', hidden_layer_sizes=(100,100,100))

resLIN, _ = SCRIPT.runNON_GP(ALG='LinReg')
resXGB, _ = SCRIPT.runNON_GP(ALG='XGboost')

gc.collect()

datasets/DATconcrete.csv
concrete
D= 8
Ntrain= 927
Ntest= 103
0 rep
1 rep
0 rep
1 rep
0 rep
1 rep
0 rep
1 rep
0 rep
1 rep


0

# run full GP and CPoE

In [18]:
# loop over SE kernel and FLEX kernel
for i in kks:
    SCRIPT = SCRIPT1(path+'DAT'+name+'.csv', Nreps=Nrep, name=name+'_kern'+str(i), kernelMODE=i, **args)
    resFull, pathFull = SCRIPT.runfullGP()
    gc.collect()
    resCPoE, pathCPoE = SCRIPT.runCPoE(K0, PPs, p)
    gc.collect()

datasets/DATconcrete.csv
concrete_kern1
D= 8
Ntrain= 927
Ntest= 103
0 rep
1 rep
0 rep
1 rep
datasets/DATconcrete.csv
concrete_kern3
D= 8
Ntrain= 927
Ntest= 103
0 rep
1 rep
0 rep
1 rep


# reload results

In [19]:
# reload results
resMLP1_load = pickle.load( open( path_results+name+'_MLP100_100', 'rb' ) )
resMLP2_load = pickle.load( open( path_results+name+'_MLP500_500', 'rb' ) )
resMLP3_load = pickle.load( open( path_results+name+'_MLP100_100_100', 'rb' ) )
resLinReg_load = pickle.load( open( path_results+name+'_LinReg', 'rb' ) )
resXGboost_load = pickle.load( open( path_results+name+'_XGboost', 'rb' ) )

In [21]:
resFull_loads = [ pickle.load( open( path_results+name+'_kern'+str(i)+'_fullGP', 'rb' ) ) for i in kks ]
resCPoE_loads1 =  [pickle.load( open( path_results+name+'_kern'+str(kks[0])+'_CPoE_K'+str(K0)+'_P'+str(P)+'_p1', 'rb' ) ) for P in PPs]
resCPoE_loads3 =  [pickle.load( open( path_results+name+'_kern'+str(kks[1])+'_CPoE_K'+str(K0)+'_P'+str(P)+'_p1', 'rb' ) ) for P in PPs]


In [22]:
# compute mean and std over the repetitions
Mmlp1, SDml1, SDMmlp1 = sdPD(resMLP1_load)
Mmlp2, SDml2, SDMmlp2 = sdPD(resMLP2_load)
Mmlp3, SDml3, SDMmlp3 = sdPD(resMLP3_load)

Mxgboost, _, SDMxgboost = sdPD(resXGboost_load)
Mlr, _, SDMlr = sdPD(resLinReg_load)

In [23]:
Mfull = pd.concat([ meanPD(x) for x in resFull_loads])
SDMfull = pd.concat([ sdmPD(x) for x in resFull_loads])

In [24]:
Mcpoe1 = pd.concat([ meanPD(x) for x in resCPoE_loads1])
SDMcpoe1 = pd.concat([ sdmPD(x) for x in resCPoE_loads1])

Mcpoe3 = pd.concat([ meanPD(x) for x in resCPoE_loads3])
SDMcpoe3 = pd.concat([ sdmPD(x) for x in resCPoE_loads3])

In [25]:
# make nice output of all results
MMM = pd.concat([Mfull, Mcpoe1,Mcpoe3, Mmlp1, Mmlp2, Mmlp3, Mxgboost, Mlr])
SDMMM = pd.concat([SDMfull, SDMcpoe1,SDMcpoe3, SDMmlp1,SDMmlp2, SDMmlp3, SDMxgboost,SDMlr])   

# rename and round
MMM.columns = SDMMM.columns = np.array(['time', 'LML', 'KL','ERR', 'CRPS', 'RMSE', 'ABSE', 'NLP', 'COV'])
dictA = {'time': 2, 'LML': 1, 'KL':1, 'ERR':3, 'CRPS':3, 'RMSE':3, 'ABSE':3, 'NLP':2, 'COV':2}
MMMr = MMM.round(dictA)
SDMMMr = SDMMM.round(dictA)

FF = MMMr.applymap(str) + ' $\pm$ '+ SDMMMr.applymap(str)
FF.index = ['fullGP-SE', 
            'fullGP-FLEX',
            \
            'CPoE(1)-SE','CPoE(2)-SE','CPoE(3)-SE','CPoE(4)-SE',
           'CPoE(1)-FLEX','CPoE(2)-FLEX','CPoE(3)-FLEX','CPoE(4)-FLEX',
           'MLP(100-100)','MLP(500-500)','MLP(100-100-100)','XGboost', 'LinReg']
GG = FF[['time','RMSE','ABSE','CRPS','COV']]
GG

,time,RMSE,ABSE,CRPS,COV
fullGP-SE,13.17 $\pm$ 2.63,0.301 $\pm$ 0.004,0.216 $\pm$ 0.005,0.162 $\pm$ 0.002,0.93 $\pm$ 0.01
fullGP-FLEX,16.83 $\pm$ 0.23,0.268 $\pm$ 0.001,0.176 $\pm$ 0.004,0.134 $\pm$ 0.003,0.93 $\pm$ 0.01
CPoE(1)-SE,2.28 $\pm$ 0.04,0.376 $\pm$ 0.008,0.256 $\pm$ 0.006,0.191 $\pm$ 0.004,0.92 $\pm$ 0.0
CPoE(2)-SE,2.76 $\pm$ 0.01,0.36 $\pm$ 0.008,0.25 $\pm$ 0.012,0.187 $\pm$ 0.006,0.9 $\pm$ 0.0
CPoE(3)-SE,3.3 $\pm$ 0.02,0.351 $\pm$ 0.009,0.246 $\pm$ 0.011,0.184 $\pm$ 0.006,0.91 $\pm$ 0.0
CPoE(4)-SE,3.35 $\pm$ 0.1,0.348 $\pm$ 0.01,0.246 $\pm$ 0.01,0.184 $\pm$ 0.006,0.9 $\pm$ 0.0
CPoE(1)-FLEX,15.93 $\pm$ 1.09,0.29 $\pm$ 0.003,0.184 $\pm$ 0.006,0.14 $\pm$ 0.005,0.93 $\pm$ 0.01
CPoE(2)-FLEX,16.49 $\pm$ 1.15,0.284 $\pm$ 0.0,0.184 $\pm$ 0.005,0.138 $\pm$ 0.003,0.94 $\pm$ 0.0
CPoE(3)-FLEX,16.84 $\pm$ 1.23,0.276 $\pm$ 0.002,0.182 $\pm$ 0.005,0.137 $\pm$ 0.004,0.94 $\pm$ 0.0
CPoE(4)-FLEX,17.8 $\pm$ 0.98,0.276 $\pm$ 0.004,0.182 $\pm$ 0.006,0.137 $\pm$ 0.004,0.94 $\pm$ 0.0


In [ ]:
#(results shown here for Nrep=2, in the paper we use Nrep=5)

# stochastic optimization for bigger datasets

In [26]:
# number of repetitions of training/test data splits
Nrep = 2        # in the paper we used Nrep = 5, which takes quite some time

In [29]:
# choose name of dataset
dataset_names = ['kin8nm', 'cadata', 'sarcos', 'casp']
name = dataset_names[1]

In [30]:
# path to datasets (run download_data.ipynb before) and location to store the results
path = 'datasets/'
path_results = 'results/'

In [37]:
# set parameters for each dataset

# sparsity parameter
p = 1    

# degree of correlation
PPs = [0,1,2]
# run non-GP methods
# K0: number of experts
# Nepochs: number of epochs of stochastic optimization
# gamma: learning rate in SGD (adam) optimization

if name=='kin8nm':
    K0 = 2**4
    Nepochs = 15
    gamma = 0.03
if name=='cadata':
    K0 = 2**5
    Nepochs = 15
    gamma = 0.01
if name=='sarcos':
    K0 = 2**7
    Nepochs = 10
    gamma = 0.01
if name=='casp':
    K0 = 2**7
    Nepochs = 10
    gamma = 0.01
args = {'path_results':path_results}

# run non-GP methods

In [33]:
SCRIPT = SCRIPT1(path+'DAT'+name+'.csv', Nreps=Nrep, name=name+'S', **args)

resMLP, _ = SCRIPT.runNON_GP(ALG='MLP', nam='100_100', hidden_layer_sizes=(100,100))
resMLP, _ = SCRIPT.runNON_GP(ALG='MLP', nam='500_500', hidden_layer_sizes=(500,500))
resMLP, _ = SCRIPT.runNON_GP(ALG='MLP', nam='100_100_100', hidden_layer_sizes=(100,100,100))
gc.collect()

datasets/DATcadata.csv
cadataS
D= 8
Ntrain= 19640
Ntest= 1000
0 rep
1 rep
0 rep
1 rep
0 rep
1 rep


0

In [34]:
resLIN, _ = SCRIPT.runNON_GP(ALG='LinReg')
resXGB, _ = SCRIPT.runNON_GP(ALG='XGboost')
gc.collect()

0 rep
1 rep
0 rep
1 rep


0

# run full GP and CPoE

In [ ]:
for i in [1,2]: # loop over SE and FLEX kernel
    SCRIPT = SCRIPT1(path+'DAT'+name+'.csv', Nreps=Nrep, FULL=False, \
                     name=name+'_kern'+str(i), kernelMODE=i, **args)
    _ = SCRIPT.runCPoE(K0, PPs, p, HYPERS='STOCH',TRACE=False, gamma=gamma, E=Nepochs)
    gc.collect()

'Epoch 14 likelihood: -13644.761643601167 rel: 0.019655078113816303 stop?: False'

 ../source/CPoE.py:379: RuntimeWarning:invalid value encountered in sqrt


# reload results

In [ ]:
# reload computed results
resMLP1_load = pickle.load( open( path_results+name+'S'+'_MLP100_100', 'rb' ) )
resMLP2_load = pickle.load( open( path_results+name+'S'+'_MLP500_500', 'rb' ) )
resMLP3_load = pickle.load( open( path_results+name+'S'+'_MLP100_100_100', 'rb' ) )
resLinReg_load = pickle.load( open( path_results+name+'S'+'_LinReg', 'rb' ) )
resXGboost_load = pickle.load( open( path_results+name+'S'+'_XGboost', 'rb' ) )

In [ ]:
resCPoE_loads1 =  [pickle.load( open( path_results+name+'_kern'+str(1)+'_CPoE_K'+str(K0)+'_P'+str(P)+'_p1stoch', 'rb' ) ) for P in PPs]
resCPoE_loads3 =  [pickle.load( open( path_results+name+'_kern'+str(2)+'_CPoE_K'+str(K0)+'_P'+str(P)+'_p1stoch', 'rb' ) ) for P in PPs]

In [ ]:
# compute mean and std over the repetitions
Mmlp1, SDml1, SDMmlp1 = sdPD(resMLP1_load)
Mmlp2, SDml2, SDMmlp2 = sdPD(resMLP2_load)
Mmlp3, SDml3, SDMmlp3 = sdPD(resMLP3_load)

Mxgboost, _, SDMxgboost = sdPD(resXGboost_load)
Mlr, _, SDMlr = sdPD(resLinReg_load)

In [ ]:
Mcpoe1 = pd.concat([ meanPD(x) for x in resCPoE_loads1])
SDMcpoe1 = pd.concat([ sdmPD(x) for x in resCPoE_loads1])

Mcpoe3 = pd.concat([ meanPD(x) for x in resCPoE_loads3])
SDMcpoe3 = pd.concat([ sdmPD(x) for x in resCPoE_loads3])

In [ ]:
# make nice output of all results
MMM = pd.concat([Mcpoe1,Mcpoe3, Mmlp1, Mmlp2, Mmlp3, Mxgboost, Mlr]).drop(['KLx1000','errFull'],axis=1)
SDMMM = pd.concat([SDMcpoe1,SDMcpoe3, SDMmlp1,SDMmlp2, SDMmlp3, SDMxgboost,SDMlr]).drop(['KLx1000','errFull'],axis=1)   

# rename and round
MMM.columns = SDMMM.columns = np.array(['time', 'LML', 'CRPS', 'RMSE', 'ABSE', 'NLP', 'COV'])
dictA = {'time': 2, 'LML': 1, 'CRPS':3, 'RMSE':3, 'ABSE':3, 'NLP':2, 'COV':2}
MMMr = MMM.round(dictA)
SDMMMr = SDMMM.round(dictA)

FF = MMMr.applymap(str) + ' $\pm$ '+ SDMMMr.applymap(str)
FF.index = ['CPoE(1)-SE','CPoE(2)-SE','CPoE(3)-SE',
           'CPoE(1)-FLEX','CPoE(2)-FLEX','CPoE(3)-FLEX',
           'MLP(100-100)','MLP(500-500)','MLP(100-100-100)','XGboost', 'LinReg']

GG = FF[['time','RMSE','ABSE','CRPS','COV']]
GG